# **Signa_Lab ITESO:** Generador de *Embbeddings*

## **Cuaderno 01:** Limpieza y depuración de texto para procesar *embeddings* desde cuerpos de texto.

Cuaderno de código abierto diseñado para importar cualquier cuerpo de texto separado por filas, en formato CSV o Excel, limpiarlo (*stopwords*, URLs, usuarios y hashtags) y depurarlo desde [diccionarios personalizados](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=drive_link) (opcional) para optimizar la posterior generación de incrustaciones de texto (*embeddings*) de cada fila ([ver cuaderno 02](https://github.com/signalab/generador-embeddings/blob/main/cuadernos/02_Signa_Lab_generador_embeddings_Generar_procesar_reducir_clusterizar_embeddings_01.ipynb)), con ayuda de modelos de lenguaje de la librería [sentence-transformers](https://www.sbert.net/), alojados en repositorios de [HuggingFace](https://huggingface.co/sentence-transformers) (en la nube) o descargados localmente.



## 1. Importar librerías y archivos de datos

### Instalar e importar librerías:

**Instalar librerías necesarias**

In [ ]:
# Instalar librerías de Python necesarias

!pip install pandas
!pip install nltk
!pip install difflib
!pip install matplotlib
!pip install seaborn
!pip install scipy
!pip install numpy
!pip install plotly
!pip install time
!pip install tqdm
!pip install operator


**Importar librerías** necesarias:

In [ ]:
# Importar librerías de Python necesarias

import pandas as pd
import nltk
import re
import sys
import re
from difflib import SequenceMatcher
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.stats import gaussian_kde
import numpy as np
import plotly.express as px
import time
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import math
import operator
from IPython.display import display, clear_output
import ipywidgets as widgets

### Indicar rutas de archivos de datos a importar y nombre de proyecto:

**Importar y concatenar archivos de datos** (en CSV o Excel):

In [ ]:
# Definir función para cargar archivos a partir de la extensión en su ruta indicada
def load_file(path):
    if path.endswith('.csv'):
        return pd.read_csv(path)
    elif path.endswith('.xlsx'):
        return pd.read_excel(path)
    else:
        raise ValueError("Formato no compatible. Por favor carga solo archivos .csv or .xlsx.")

# Inicializar lista para alojar todas las rutas y una variable para el DataFrame final, accesible globalmente
file_paths = []
dfs = []
df = None  # DataFrame global

# Definir función para añadir un nuevo campo de texto (input) para añadir una ruta de archivo adicional
def add_file_input(b=None):
    path_input = widgets.Text(value='', placeholder='Escribe la ruta del archivo', description=f'File {len(file_paths) + 1}:')
    file_paths.append(path_input)
    update_ui()

# Definir función para eliminar el último campo de texto (input) para ruta de archivo
def remove_file_input(b=None):
    if file_paths:
        file_paths.pop()
        update_ui()

# Definir función para procesar y cargar todos los archivos
def process_files(b):
    global dfs, df
    dfs = []  # Vaciar DataFrames

    for path_input in file_paths:
        path = path_input.value
        try:
            temp_df = load_file(path)
            temp_df['filename'] = path  # Add a column with the filename
            dfs.append(temp_df)
            print(f"Nombre de archivo: {path}")
            print(f"Filas/Columnas (shape): {temp_df.shape}")
        except ValueError as e:
            print(f"Error al cargar el archivo {path}: {e}")
            return

    if dfs:
        df = pd.concat(dfs, ignore_index=True)  # Concatenate all DataFrames
        print("\n¡Se cargaron todos los archivos!")
        print(f"Filas/Columnas (shape) de DataFrame creado: {df.shape}")

# Campo de texto (input) para indicar nombre del proyecto (para integrarse en nombres de archivos a exportar)
project_name = widgets.Text(value='', placeholder='Escribe el nombre del proyecto (corto y sin espacios)', description='Nombre del proyecto:')

# Botones para añadir y eliminar archivos
add_button = widgets.Button(description="Añadir archivo")
remove_button = widgets.Button(description="Eliminar archivo")
load_button = widgets.Button(description="Cargar archivos")

add_button.on_click(add_file_input)
remove_button.on_click(remove_file_input)
load_button.on_click(process_files)

# Definir función para actualizar UI
def update_ui():
    clear_output()
    display(project_name)
    for path_input in file_paths:
        display(path_input)
    display(widgets.HBox([add_button, remove_button]))
    display(load_button)

# Inicializar UI con un campo de texto (input) para ruta de archivo
add_file_input()


### Previsualizar datos importados:

**Previsualizar tabla** con todos los registros importados:

In [ ]:
# Previsualizar dataframe con CSVs importados
display(df)
print(f"Filas/Columnas (shape) en registros importados: {df.shape}")


**Exportar copia en CSV con registros importados (concatenados):**

In [ ]:
# Exportar archivo CSV con tabla de registros importados (y concatenados, en el caso de múltiples archivos)
df.to_csv(f"{project_name.value}_registros-importados.csv")

---

## 2. Limpieza de registros importados

### Generar identificadores únicos (IDs) por registro:

In [ ]:
# Definir función para asignar IDs únicos a cada fila en el data frame indicado como parámetro, comenzando desde '1000001'.
def assign_unique_ids(df):
    # Inicializar contador para IDs
    id_counter = 1000001

    # Crear copia de data frame original
    df_copy = df.copy()

    # Iterar a través de las filas del data frame
    for index, _ in enumerate(df_copy.index):
        # Dar formato a ID con ceros adicionales e incorporarlo al data frame
        formatted_id = str(id_counter).zfill(7)  # Se asegura de que sea un ID de 7 dígitos, agregando ceros cuando sea necesario
        df_copy.loc[index, 'id'] = formatted_id

        # Incrementar el contador del ID para la siguiente iteración
        id_counter += 1

    return df_copy

In [ ]:
# Ejecutar función para asignar IDs a cada registro y previsualizar tabla
if __name__ == "__main__":
    # Invocar la función con data frame de trabajo
    df_ids = assign_unique_ids(df)

# Sobreescribir data frame con nueva tabla con IDs generados
df = df_ids
df

In [ ]:
# Revisar el número de filas y columnas en tabla de registros con IDs generados
df.shape

### Limpieza de texto sin aporte semántico:

In [ ]:
# Definir función para limpiar usuarios, hashtags y URLs
def limpiar_texto(texto):
  # Eliminar usuarios (opcional, comentar siguiente línea para omitirlo)
  # texto = re.sub(r"(?<!\w)@(\w+)(?!\w)", "", texto)

  # Eliminar hashtags (opcional, comentar siguiente línea para omitirlo)
  # texto = re.sub(r"(?<!\w)#(\w+)(?!\w)", "", texto)

  # Eliminar URLs (opcional, comentar siguiente línea para omitirlo)
  texto = re.sub(r"(http|https|ftp)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?", "", texto)
  texto = texto.lstrip(". ")

  return texto.strip()

**Ejecutar funciones y agregar nueva columna con registros con texto limpio:**

In [ ]:
# Definir función para agregar nueva columna con texto limpio (clean_text)
def agregarCleanTextADf(df, colText):
    dfW = df.copy()
    dfW["clean_text"] = None

    for index, row in dfW.iterrows():
        text = str(row[colText])
        cleaned_text = limpiar_texto(text)
        dfW.at[index, "clean_text"] = cleaned_text

    return dfW

In [ ]:
# Especificar nombre de columna con texto para limpiar y procesar embeddings
# text_column = "nombre-columna-texto"

text_column = "title"


# Ejecutar limpieza de usuarios, hashtags y URLs
dfCleanText = agregarCleanTextADf(df, text_column)

In [ ]:
# Previsualizar tabla con nueva columna de datos limpios (clean_text)
dfCleanText.head()

### Eliminar palabras vacías (*stop words*):

Descargar dependencias de librería NLTK para **limpieza de palabras vacías (*stop words*):**

In [ ]:
# Descargar las stop words en español e inglés
nltk.download('punkt')
nltk.download('stopwords')

# Lista de stopwords en español
stopwords_es = nltk.corpus.stopwords.words('spanish')
# stopwords_en = nltk.corpus.stopwords.words('english')

**Función para eliminar palabras vacías (*stop words*)**

In [ ]:
# Definir función para eliminar palabras vacías (stopwords) y signos
def delete_stopwords(texto):

  # Tokenizar el texto
  tokens = nltk.word_tokenize(texto)

  # Eliminar signos de puntuación
  tokens = [token for token in tokens if token.isalnum()]

  # Eliminar stop words
  tokens = [token for token in tokens if token not in stopwords_es]

  # Convertir la lista de tokens a un string
  texto_limpio = " ".join(tokens)

  return texto_limpio.strip()

Ejecutar funciones y **agregar columna con texto sin *stop words* (sem_text):**

In [ ]:
# Definir función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
def agregarSemTextADf(df, colText):
    dfW = df.copy()
    dfW["sem_text"] = None

    for index, row in dfW.iterrows():
        text = row[colText]
        cleaned_text = delete_stopwords(text)
        dfW.at[index, "sem_text"] = cleaned_text

    return dfW

In [ ]:
# Ejecutar función para eliminar palabras vacías y agregar nueva columna con el resultado (sem_text)
dfClean = agregarSemTextADf(dfCleanText, "clean_text")

In [ ]:
# Previsualizar tabla de datos con nueva columna de registros con texto sin palabras vacías (sem_text)
dfClean.head()

In [ ]:
# Revisar el número de filas y columnas en tabla de registros con texto sin palabras vacías (sem_text)
dfClean.shape

In [ ]:
# Exportar archivo CSV con tabla completa de registros importados con IDs y texto sin palabras vacías (sem_text)
dfClean.to_csv(f"{project_name.value}_registros-semtext.csv")

## 3. Depuración de registros desde diccionarios personalizados (con términos de filtrado o descarte)







### Definir uso de de diccionario de depuración:

A partir de diccionarios personalizados con términos de descarte o filtración, elige depurar o mantener, respectivamente, registros que los contengan.

El diccionario debe cargarse en formato CSV y contener, al menos, los siguientes campos:

| palabra | tipo | categoría | diccionario |
|---------|------|-----------|-------------|
|         |      |           |             |
|         |      |           |             |
|         |      |           |             |

Ejemplo:

| palabra | tipo   | categoría   | diccionario        |
|---------|--------|-------------|--------------------|
| idiota  | ofensa | humillación | ofensa-humillación |
| zorra   | ofensa | género      | ofensa-género      |
|         |        |             |                    |

Puedes [encontrar aquí una copia del diccionario](https://drive.google.com/file/d/1zK214W0pBRYn9lnY_MDJYhEEc6pI3L6F/view?usp=sharing) con la estructura requerida, en formato CSV, para descargar, llenar e incorporar a este cuaderno de código.

**Nota:** La versión actual de este cuaderno de código permite elegir entre las siguientes opciones sobre la carga de diccionarios:


- ***No cargar diccionario:*** Ignorar esta funcionalidad y no usar diccionarios.
- ***Diccionario de descarte (negativo):*** Se eliminarán todos los registros que mencionen alguno de esos términos.
- ***Diccionario de filtrado (positivo):*** Se mantendrán solo los registros que mencionen alguno de esos términos.

In [ ]:
# Create the dropdown widget for dictionary usage selection
cargar_diccionario = widgets.Dropdown(
    options=["No cargar diccionario", "Diccionario de descarte (negativo)", "Diccionario de filtrado (positivo)"],
    value="No cargar diccionario",
    description="Elegir uso:"
)

# Create the text input widget for file path
rutaDicc = widgets.Text(
    value="",
    placeholder="Escribir ruta a archivo CSV",
    description="Ruta:"
)

# Create the "Aceptar" button
accept_button = widgets.Button(description="Aceptar")

# Create an output widget to display results or errors
output = widgets.Output()

# Function to load the dictionary based on user input
def load_dictionary(b):
    with output:
        clear_output()
        if cargar_diccionario.value != "No cargar diccionario":
            try:
                global dfDiccionario
                dfDiccionario = pd.read_csv(rutaDicc.value)
                print(f"Archivo de diccionario '{rutaDicc.value}' cargado exitosamente.")
                display(dfDiccionario.head())  # Show the first few rows of the loaded dictionary
            except FileNotFoundError:
                print(f"El archivo de diccionario '{rutaDicc.value}' no fue encontrado. Por favor, verifique la ruta.")
        else:
            dfDiccionario = None
            print("No se cargó ningún diccionario.")

# Link the button to the load function
accept_button.on_click(load_dictionary)

# Display the widgets
display(cargar_diccionario, rutaDicc, accept_button, output)


In [ ]:
# Verificar uso de diccionario elegido
print(f"Uso de diccionario elegido: {cargar_diccionario.value}")

### Aplicar filtrado por diccionario y previsualizar resultados (opcional, solo si se cargó algún diccionario):

**Previsualizar diccionario de descarte importado:**

In [ ]:
if cargar_diccionario.value != "No cargar diccionario":
  # Previsualizar tabla de diccionario cargado (en caso de haber elegido utilizar diccionarios)
  try:
      display(dfDiccionario.head())  # Previsualizar primeros registros de diccionario
      print(f"\nFilas y columnas en diccionario cargado:")  # Verificar número de filas y columnas en diccionario
      print(f"Shape: {dfDiccionario.shape}")  # Verificar número de filas y columnas en diccionario
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")


**Filtrar registros a partir de diccionario cargado y modalidad de depuración (positiva o negativa):**

In [ ]:
def filtrar_registros(df_registros, df_terminos, colTexto, cargar_diccionario):
    # Definir los términos del diccionario
    terminos = df_terminos["palabra"].tolist()
    # Compilar expresiones regulares una sola vez
    expresiones_regex = [
        re.compile(r"(?<!\S)?(?:\s|[.,;:?!¡¿]){}(?:\s|[.,;:?!¡¿])?(?!\S)".format(re.escape(termino)), re.IGNORECASE)
        for termino in terminos
    ]

    # Copiar el dataframe de registros y agregar columnas auxiliares
    df_registros_filtrados = df_registros.copy()
    df_registros_filtrados["contiene_termino"] = False
    df_registros_filtrados["razon_eliminacion"] = ""

    # Filtrar registros
    for i in range(df_registros_filtrados.shape[0]):
        texto = str(df_registros_filtrados.loc[i, colTexto]).lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")

        # Buscar coincidencias con expresiones regulares
        for expresion, razon in zip(expresiones_regex, df_terminos["categoría"]):
            coincidencias = expresion.findall(f" {texto} ")
            if coincidencias:
                df_registros_filtrados.loc[i, "contiene_termino"] = True
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Presencia de términos relacionados a {razon}"
                break
            else:
                df_registros_filtrados.loc[i, "contiene_termino"] = False
                df_registros_filtrados.loc[i, "razon_eliminacion"] = f"Ausencia de términos relacionados a {razon}"

    # Revisar uso elegido del diccionario, para descarte (negativo) o filtrado (positivo)
    if cargar_diccionario == "Diccionario de descarte (negativo)":
        # Mantener registros que no contengan términos del diccionario (negativo)
        df_registros_filtrados_final = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
    elif cargar_diccionario == "Diccionario de filtrado (positivo)":
        # Mantener registros que sí contengan al menos un término del diccionario (positivo)
        df_registros_filtrados_final = df_registros_filtrados[df_registros_filtrados["contiene_termino"]]
        df_registros_eliminados = df_registros_filtrados[~df_registros_filtrados["contiene_termino"]]

    # Eliminar columnas auxiliares para tabla con registros depurados
    df_registros_filtrados_final = df_registros_filtrados_final.drop(columns=["contiene_termino", "razon_eliminacion"])

    df_registros_filtrados_final = df_registros_filtrados_final.reset_index(drop=True)
    df_registros_eliminados = df_registros_eliminados.reset_index(drop=True)

    return df_registros_filtrados_final, df_registros_eliminados


In [ ]:
# Ejecutar depuración de registros por diccionario cargado
if cargar_diccionario.value != "No cargar diccionario":
  # try:
  df_depurados_dicc, df_eliminados_dicc = filtrar_registros(dfClean, dfDiccionario, text_column, cargar_diccionario.value)
  # except NameError:
      # print("Otro error...")
else:
  print("No se cargó ningún diccionario...")

**Previsualizar registros depurados y eliminados (opcional):**

In [ ]:
# Previsualizar tabla de registros depurados (conservados) y verificar número de filas y columna
if cargar_diccionario.value != "No cargar diccionario":
  try:
    display(df_depurados_dicc.head())
    print(f"\nFilas/Columnas (shape) en registros conservados por diccionario: {df_depurados_dicc.shape}")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Previsualizar tabla de registros eliminados y verificar su número de filas y columnas
if cargar_diccionario.value != "No cargar diccionario":
  try:
    display(df_eliminados_dicc.head())
    print(f"\nFilas/Columnas (shape) en registros eliminados por diccionario: {df_eliminados_dicc.shape}")
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  print("No se cargó ningún diccionario...")

In [ ]:
# Verificar eliminación de casos específicos en registros eliminados o conservados (opcional)

# Indicar palabra a buscar
if cargar_diccionario.value != "No cargar diccionario":
  verificar_termino = "vandalismo"

  _count = 0
  # Buscar palabra en registros conservados
  for i in df_depurados_dicc['sem_text']:

  # Buscar palabra en registros eliminados
  # for i in df_eliminados_dicc['sem_text']:

      words = i.split()
      if verificar_termino in words:
          _count += 1
  print(_count)
else:
  print("No se cargó ningún diccionario...")

## 4. Revisar, contar y eliminar registros repetidos

### Definir y aplicar función para eliminar redacciones repetidas:

**Eliminar registros repetidos:**

Eliminar aquellos registros que contengan una similitud en su redacción mayor a un umbral establecido (por default asignado al 100%, con valor de 1), para así buscar eliminar registros con una repetición exacta.


In [ ]:
# Definir función para calcular la similitud entre dos listas de palabras
def Similarity_Score(list1, list2):
    # Inicializar contadores para coincidencias y longitud total
    matches = 0
    total_length = 0

    # Iterar sobre las listas hasta el tamaño de la lista más corta
    for i in range(min(len(list1), len(list2))):
        # Si las palabras en las mismas posiciones coinciden, incrementar el contador de coincidencias
        if list1[i] == list2[i]:
            matches += 1
        # Incrementar el contador de longitud total
        total_length += 1

    # Para las posiciones adicionales en la lista más larga, incrementar el contador de longitud total
    for i in range(min(len(list1), len(list2)), max(len(list1), len(list2))):
        total_length += 1

    # Calcular el ratio de coincidencias como la proporción de coincidencias sobre la longitud total
    ratio = matches / total_length

    return ratio

**Definir función para identificar registros repetidos:**

In [ ]:
def remove_duplicates_with_threshold(df, column, threshold):
    global similarity_score
    print("Se actualizó")
    indices_to_remove = set()
    sentence_frequency = defaultdict(int) # Diccionario para almacenar la frecuencia de registros similares
    discarded_info = defaultdict(list) # Diccionario para almacenar información de registros descartados

    # Crear índice invertido para las palabras en los registros
    inverted_index = defaultdict(set)
    for i, sentence in enumerate(df[column]):
        words = set(sentence.split())
        for word in words:
            inverted_index[word].add(i)

    print(f"{len(df[column])} registros en total")
    for i, sentence in enumerate(df[column]):
        if i not in indices_to_remove:
            similar_sentences_count = 1 # Contador de registros similares para la fila actual
            words = set(sentence.split())
            relevant_indices = set()
            for word in words:
                relevant_indices |= inverted_index[word]

            for j in relevant_indices:
                if j != i and j not in indices_to_remove:
                    registroSinAcentos = sentence.replace('á', 'a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
                    registroSinAcentosEnLista = registroSinAcentos.split(" ")
                    registroAComparar = df[column][j]
                    registroACompararSinAcentos = registroAComparar.replace('á', 'a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
                    registroACompararEnLista = registroACompararSinAcentos.split(" ")

                    score = Similarity_Score(list(registroSinAcentosEnLista), list(registroACompararEnLista))
                    if score >= threshold:
                        indices_to_remove.add(j)
                        similar_sentences_count += 1
                        # Almacenar información de la registro descartado
                        discarded_info[j].append({'original_index': df['id'][i], 'similarity_score': score})
            sentence_frequency[i] = similar_sentences_count # Almacenar la frecuencia de registros similares para la fila actual

        if i % 1000 == 0:
            print(f"Van {i} registros revisados...")

    # Eliminar los registros duplicados después de completar el bucle
    filtered_df = df.drop(indices_to_remove).reset_index(drop=True)

    # Crear DataFrame con registros duplicados
    df_removed_duplicates = df.iloc[list(indices_to_remove)]

    # Agregar información de registros descartados al DataFrame de registros descartados
    id_match = []
    similarity_score = []

    # Iterar sobre índice de DataFrame
    for index in df_removed_duplicates.index:
        # Revisar si el índice se encuentra en discarded_info
        if index in discarded_info:
            # Por cada índice, toma el primer elemento de 'original_index' y 'similarity_score'
            id_match.append(discarded_info[index][0]['original_index'])
            similarity_score.append(discarded_info[index][0]['similarity_score'])
        else:
            # Si el índice no está en discarded_info, agregar el valor por default None
            id_match.append(None)
            similarity_score.append(None)

    df_removed_duplicates['id_match'] = id_match
    df_removed_duplicates['similarity_score'] = similarity_score

    filtered_df['sentence_frequency_count'] = filtered_df['id'].apply(
        lambda x: len(df_removed_duplicates[df_removed_duplicates['id_match'] == x]) + 1)

    return filtered_df, df_removed_duplicates

Ejecutar función para **eliminar registros duplicados**

In [ ]:
# Establecer umbral de similitud (porcentaje 0 a 100%, indicando un valor de 0 a 1)
threshold = 1 # Con 1, se eliminan registros que sean 100% similares en su redacción a alguno ya registrado
# Ejecutar elminación de registros repetidos

if cargar_diccionario.value != "No cargar diccionario":
  df_depurados_final, df_removed_duplicates = remove_duplicates_with_threshold(df_depurados_dicc, 'sem_text', threshold)
else:
  df_depurados_final, df_removed_duplicates = remove_duplicates_with_threshold(dfClean, 'sem_text', threshold)
  print("No se cargó ningún diccionario...")

### Previsualizar resultados de conteo y depuración de registros repetidos:

In [ ]:
# Previsualizar tabla de población de registros depurados con la frecuencia de aparición de cada registro respecto a otros registros
df_depurados_final.head()

In [ ]:
# Definir función para agregar la razón de eliminación por repetidos
def agregar_razon_eliminacion(df_removed, razon):
    df_removed['razon_eliminacion'] = razon
    return df_removed

df_removed_duplicates = agregar_razon_eliminacion(df_removed_duplicates, 'Redacción repetida respecto a otro registro')

In [ ]:
# Previsualizar tabla de registros eliminados por repetición con razón de eliminación
df_removed_duplicates

In [ ]:
# Revisar el número de filas y columnas en tabla de registros eliminados por repetición
df_removed_duplicates.shape

## 5. Revisar y exportar datos con registros depurados y eliminados

### Consolidar y previsualizar tablas finales de registros depurados y eliminados:

**Consolidar tabla de registros eliminados, concatenando tablas de eliminados por diccionario (en caso de haberse cargado) y por repeticiones:**

In [ ]:
if cargar_diccionario.value != "No cargar diccionario":
  try:
    # Ejecutar concatenación de filas eliminadas por términos en diccionario y repeticiones
    df_eliminados_final = pd.concat([df_eliminados_dicc, df_removed_duplicates], axis=0)
  except NameError:
      print("No se cargó ningún diccionario...")
else:
  df_eliminados_final = df_removed_duplicates

In [ ]:
# Previsualizar tabla de registros totales eliminados (por términos en diccionario y repeticiones), con razón de eliminación
display(df_eliminados_final)
print(f"Filas/Columnas (shape) en resgistros totales eliminados: {df_eliminados_final.shape}")


In [ ]:
# Previsualizar tabla de registros totales depurados (conservados)
display(df_depurados_final)
print(f"Filas/Columnas (shape) en registros totales conservados: {df_depurados_final.shape}")


### Exportar archivos CSV con tablas finales de registros depurados y elminados:

**Exportar archivo de datos (en formato CSV) de población de registros depurados a utilizar:**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con población de registros depurados
df_depurados_final.to_csv(f"{project_name.value}_poblacion-registros-depurados.csv")

print(f"¡{project_name.value}_poblacion-registros-depurados.csv descargado!")

**Exportar archivo de datos (en formato CSV) de registros eliminados por términos de descarte o repeticiones, con su razonamiento correspondiente:**

In [ ]:
# Ejemplo exportar archivo de datos (CSV) con registros eliminados por términos de descarte en diccionarios y repeticiones
df_eliminados_final.to_csv(f"{project_name.value}_registros-eliminados.csv")

print(f"¡{project_name.value}_registros-eliminados.csv descargado!")

## 6. Referencias

*   Bird, Steven, Edward Loper & Ewan Klein (2009).
Natural Language Processing with Python.  O'Reilly Media Inc.
* Kiss, T., & Strunk, J. (2006). Unsupervised Multilingual Sentence Boundary Detection. Computational Linguistics, 32(4), 485-525. https://doi.org/10.1162/coli.2006.32.4.485

## 7. Créditos

**Realizado por el equipo de Signa_Lab ITESO:**

- **Programación de cuadernos de código (Python)**:
Javier de la Torre Silva, José Luis Almendarez González y Diego Arredondo Ortiz

- **Supervisión del desarrollo tecnológico y documentación:**
Diego Arredondo Ortiz

- **Equipo de Coordinación Signa_Lab ITESO:**
Paloma López Portillo Vázquez, Víctor Hugo Ábrego Molina y Eduardo G. de Quevedo Sánchez

Mayo, 2024. Instituto Tecnológico y de Estudios Superiores de Occidente (ITESO)
Tlaquepaque, Jalisco, México.


---